![logo.png](./img/logo.png)

## Open Policy Agent

A general purpose policy engine

With a declarative language: Rego

### Background

Started by Styra in late 2015

![commits.png](./img/commits.png)

Current version: 1.15.1

Adopted into the Cloud Native Computing Foundation in March 2018

Currently in the Incubating stage

In the Assess stage of Tech Radar (73)

### General workings

Most of the following from Red Hat's blog [here](https://servicesblog.redhat.com/2019/10/16/open-policy-agent-part-i-the-introduction/).

![flow.png](./img/flow.png)

In [1]:
ls src/

acl.json  policy.rego


#### Our access control list:

In [12]:
cat src/acl.json

{
  "nathan": [
    "read"
  ],
  "bryan": [
    "read"
  ],
  "courtney": [
    "read",
    "write"
  ],
  "kurt": [
    "read"
  ]
}

#### Our policy definition:

In [15]:
cat src/policy.rego

package myapi.policy

import data.myapi.acl
import input

default allow = false

allow {
        access = acl[input.user]
        access[_] == input.access
}

whocan[user] {
        access = acl[user]
        access[_] == input.access
}

#### The cli:

In [17]:
opa

An open source project to policy-enable your service.

Usage:
  opa [command]

Available Commands:
  build       Compile Rego policy queries
  check       Check Rego source files
  deps        Analyze Rego query dependencies
  eval        Evaluate a Rego query
  fmt         Format Rego source files
  help        Help about any command
  parse       Parse Rego source file
  run         Start OPA in interactive or server mode
  test        Execute Rego test cases
  version     Print the version of OPA

Flags:
  -h, --help   help for opa

Use "opa [command] --help" for more information about a command.


#### Start up our server:

In [18]:
opa run --server &

[1] 23084


: 1

#### Upload our acl and policy:

In [32]:
curl -X PUT http://localhost:8181/v1/data/myapi/acl \
--data-binary @src/acl.json

{"client_addr":"[::1]:52536","level":"info","msg":"Received request.","req_id":14,"req_method":"PUT","req_path":"/v1/data/myapi/acl","time":"2019-12-08T01:12:40-05:00"}
{"client_addr":"[::1]:52536","level":"info","msg":"Sent response.","req_id":14,"req_method":"PUT","req_path":"/v1/data/myapi/acl","resp_bytes":0,"resp_duration":0.340984,"resp_status":204,"time":"2019-12-08T01:12:40-05:00"}


In [33]:
curl -X PUT http://localhost:8181/v1/policies/myapi \
--data-binary @src/policy.rego

{"client_addr":"[::1]:52546","level":"info","msg":"Received request.","req_id":15,"req_method":"PUT","req_path":"/v1/policies/myapi","time":"2019-12-08T01:12:49-05:00"}
{"client_addr":"[::1]:52546","level":"info","msg":"Sent response.","req_id":15,"req_method":"PUT","req_path":"/v1/policies/myapi","resp_bytes":2,"resp_duration":4.555858,"resp_status":200,"time":"2019-12-08T01:12:49-05:00"}
{}

#### And test:

In [36]:
curl -s -X POST http://localhost:8181/v1/data/myapi/policy/allow \
--data-binary '{ "input": { "user": "courtney", "access": "write" } }' | jq

{"client_addr":"[::1]:52576","level":"info","msg":"Received request.","req_id":18,"req_method":"POST","req_path":"/v1/data/myapi/policy/allow","time":"2019-12-08T01:13:16-05:00"}
{"client_addr":"[::1]:52576","level":"info","msg":"Sent response.","req_id":18,"req_method":"POST","req_path":"/v1/data/myapi/policy/allow","resp_bytes":15,"resp_duration":2.616285,"resp_status":200,"time":"2019-12-08T01:13:16-05:00"}
{
  "result": true
}


#### Cleanup:

In [37]:
pkill opa

{"level":"info","msg":"Shutting down...","time":"2019-12-08T01:13:44-05:00"}
{"level":"info","msg":"Server shutdown.","time":"2019-12-08T01:13:44-05:00"}


: 1

## Other uses

Adding user auth to the Docker daemon ([link](https://www.openpolicyagent.org/docs/latest/docker-authorization/))

Adding a policy to Terraform ([link](https://www.openpolicyagent.org/docs/latest/terraform/))

Only allowing images from a trusted registry in Kubernetes

### Future

Gatekeeper v3 in beta currently

WebAssembly compiler going GA in v0.16